https://mlflow.org/docs/latest/deep-learning/tensorflow/guide/index.html

In [1]:
import mlflow
import numpy as np
import tensorflow as tf
from tensorflow import keras

2024-06-25 12:16:40.216296: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


logging system metrics

In [2]:
import os

os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

defining tracking server

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-experiment")

<Experiment: artifact_location='mlflow-artifacts:/973493498494548905', creation_time=1719317263517, experiment_id='973493498494548905', last_update_time=1719317263517, lifecycle_stage='active', name='my-experiment', tags={'mlflow.sharedViewState.dcb2842cfd9de80df0723b295bd0922e8a7e1fdb7e6a73e69628028bd5d9f213': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","compareRunCharts":[{"uuid":"1719320514485vpv2smex","type":"LINE","runsCountToCompare":10,"metricSectionId":"1719320514484oc0177ek","deleted":false,"isGenerated":true,"metricKey":"loss","lineSmoothness":0,"xAxisScaleType":"linear","scaleType":"linea

logging training

In [4]:
mlflow.tensorflow.autolog()

# Prepare data for a 2-class classification.
data = np.random.uniform(size=[20, 28, 28, 3])
label = np.random.randint(2, size=20)

model = keras.Sequential(
    [
        keras.Input([28, 28, 3]),
        keras.layers.Conv2D(8, 2),
        keras.layers.MaxPool2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(2),
        keras.layers.Softmax(),
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

with mlflow.start_run():
    model.fit(data, label, batch_size=5, epochs=2)

2024-06-25 12:16:42.854928: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024/06/25 12:16:43 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/06/25 12:16:43 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/2
3/4 [=====================>........] - ETA: 0s - loss: 0.6233 - sparse_categorical_accuracy: 0.7333

2024/06/25 12:16:44 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


4/4 [==============================] - 1s 73ms/step - loss: 0.8806 - sparse_categorical_accuracy: 0.6000
Epoch 2/2
3/4 [=====================>........] - ETA: 0s - loss: 0.7571 - sparse_categorical_accuracy: 0.6000

2024/06/25 12:16:44 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


1/1 [==============================] - 0s 129ms/step


INFO:tensorflow:Assets written to: /tmp/tmpn8vsgqkh/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpn8vsgqkh/model/data/model/assets
2024/06/25 12:16:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/thiagoads/Workspace/Programas/miniconda3/envs/tensorflow-mlflow/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
2024/06/25 12:16:52 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/25 12:16:52 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


loading logged model

In [6]:
logged_model = 'runs:/a5e04936699a4f939ecb21ade4941f4e/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model.predict(data)

1/1 [==============================] - 0s 53ms/step


array([[0.50548565, 0.49451432],
       [0.30224618, 0.6977538 ],
       [0.36716378, 0.6328363 ],
       [0.3264776 , 0.6735224 ],
       [0.40671006, 0.59329   ],
       [0.4070008 , 0.59299916],
       [0.42451376, 0.5754863 ],
       [0.3491376 , 0.6508624 ],
       [0.46203443, 0.5379655 ],
       [0.4924147 , 0.5075853 ],
       [0.47422326, 0.52577674],
       [0.3902951 , 0.6097049 ],
       [0.34209004, 0.6579099 ],
       [0.33114392, 0.6688561 ],
       [0.3754936 , 0.62450635],
       [0.4359612 , 0.5640388 ],
       [0.426095  , 0.57390493],
       [0.45762205, 0.542378  ],
       [0.42206103, 0.577939  ],
       [0.36858198, 0.631418  ]], dtype=float32)